In [15]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from scipy import interpolate
from datetime import datetime
import math

In [ ]:
df= pd.read_csv('Q1_Data.csv')
df2= pd.DataFrame(columns=['Times'])
for i in range(len(df['Date'])):
    df2= df2.append({'Times' :datetime.strptime(df['Date'][i], '%m/%d/%Y')}, ignore_index=True)
df['Date']=df2
df=df.set_index('Date', drop= False)
df['Date']= (df['Date']- df['Date'][0])

In [ ]:
df.head()

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(df['Date'],df['Price'],'o-')

## First question

In [ ]:
e=0
x=[]
y=[]
for i in range(1,13):
    if df['Price'][i]>df['Price'][i+1] and df['Price'][i]>df['Price'][i-1]:
        for j in range(4):
            x.append((df['Date'][datetime(df.index[i].year+j, df.index[i].month, df.index[i].day)]).days)
            y.append(df['Price'][datetime(df.index[i].year+j, df.index[i].month, df.index[i].day)])
        f = interpolate.interp1d(x[-4:], y[-4:] ,bounds_error= False, fill_value='extrapolate')
        df= df.append(pd.Series({'Price':float( f( (datetime(df.index[i].year+j+1, df.index[i].month, df.index[i].day)- df.index[0]).days)),
                                 'Date':(datetime(df.index[i].year+j+1, df.index[i].month, df.index[i].day)- df.index[0] ) } ,
                                name= datetime(df.index[i].year+j+1, df.index[i].month, df.index[i].day)))
        e+=1
    elif df['Price'][i]<df['Price'][i+1] and df['Price'][i]<df['Price'][i-1] :
        for j in range(4):
            x.append((df['Date'][datetime(df.index[i].year+j, df.index[i].month, df.index[i].day)]).days)
            y.append(df['Price'][datetime(df.index[i].year+j, df.index[i].month, df.index[i].day)])
        f = interpolate.interp1d(x[-4:], y[-4:] ,bounds_error= False, fill_value='extrapolate')
        df= df.append(pd.Series({'Price':float( f( (datetime(df.index[i].year+j+1, df.index[i].month, df.index[i].day)- df.index[0]).days)),
                                 'Date':(datetime(df.index[i].year+j+1, df.index[i].month, df.index[i].day)- df.index[0] ) } ,
                                name= datetime(df.index[i].year+j+1, df.index[i].month, df.index[i].day)))
        e+=1
    if e==4:
        break

In [ ]:
f = interpolate.interp1d( df['Date'].dt.days ,df['Price'] ,kind='cubic',bounds_error= False, fill_value='extrapolate')
ynew= f(np.arange(0, int(df['Date'][-1:].dt.days) +1, 1))
plt.figure(figsize=(12,8))
plt.plot( df['Date'].dt.days,df['Price'], 'o', np.arange(0, int(df['Date'][-1:].dt.days) +1, 1), ynew, '-')

## Second question

In [ ]:
cost_per_day = int(input('Cost in $ for storage per day: '))
volume = int(input('Storage Volume: '))

In [ ]:
def long_term(y, cost_per_day, volume):
    return (y[-1:]-y[0])*volume - cost_per_day*len(y)*volume
long_term(ynew, cost_per_day, volume)

## Third question

In [ ]:
i= int(input('Day of injection: '))
l= int(input('Day of extraction: '))
def short_term(y,i,l, cost_per_day, volume):
    return (y[l-1]-y[i-1])*volume - cost_per_day*len(y)*volume
short_term(ynew,i,l, cost_per_day, volume)

## Fourth question

In [ ]:
rate = int(input('Rate of injection/extraction: '))
penalty = int(input('Penalty: '))
inj = input('Injection dates in mm/dd/yyy format: ').split(' ')
ext = input('Extraction dates in mm/dd/yyy format: ').split(' ')
inj_ext= np.zeros(len(y))
for i in inj:
    inj_ext[(datetime.strptime(i, '%m/%d/%Y')-df.index[0]).dt.days]=1
for i in ext:
    inj_ext[(datetime.strptime(i, '%m/%d/%Y')-df.index[0]).dt.days]= -1
def with_penalty(y, inj_ext,rate, penalty , cost_per_day, volume):
    volume_filled=0
    value=0
    for i in range(inj_ext):
        if inj_ext[i]==1:
            volume_filled+=rate
            value-= y[i]*rate*10**(-6)
        elif inj_ext[i]==-1 and volume_filled>=rate:
            volume_filled-+rate
            value+= y[i]*rate*10**(-6)
        if volume_filled > volume:
            value-= penalty 
        value-= cost_per_day*volume_filled

## Fifth Question

In [ ]:
dates= input('Possible Injection/Extraction dates in mm/dd/yyy format: ').split(' ')
drift = input().split(' ')
inj = input('Injection dates in mm/dd/yyy format: ').split(' ')
ext = input('Extraction dates in mm/dd/yyy format: ').split(' ')
sigma= int(input())
s_o = input('Price at first day ')
m= input('No. of paths: ')

In [ ]:
p_c=[]
for i in range(m):
    drift_t1 = sum([i for i in drift[:dates.index(inj)] ])
    drift_t2 = sum([i for i in drift[:dates.index(ext)] ])
    s_t1= s_o*math.exp(drift_t1 - sigma**2*dates.index(inj)/2 +sigma*np.random.normal(scale=dates.index(inj) ))
    s_t2= s_o*math.exp(drift_t2 - sigma**2*dates.index(ext)/2 +sigma*np.random.normal(scale=dates.index(ext) ))
    p_c.append((s_t2 - s_t1)*volume- (dates.index(ext)- dates.index(inj))*cost_per_day*volume )
    
print('Price of contract is :', np.mean(p_c))